In [ ]:
%pip install --upgrade pip
%pip install cmake

In [ ]:
%pip install opencv-python opencv_contrib-python
%pip install face_recognition
%pip install numpy 

In [4]:
import cv2
import face_recognition
import os, numpy as np

In [6]:
#Directory to store training images
os.makedirs('training_faces', exist_ok=True)
os.makedirs('encodings', exist_ok=True)

Save faces

In [ ]:

def capture_images():
    #Name for the captured images
    name = input("Enter name: ")

    #Create a folder for person if it doesnt exist yet
    person_folder = f"training_faces/{name}"
    os.makedirs(person_folder, exist_ok= True)


    #Initialize video capture
    cap = cv2.VideoCapture(0)

    #Frames Loop
    while True:
        success, frame = cap.read()
        if not success:
            print("Failed to capture frame")
            break

        cv2.imshow("Training - Press 'c' to capture, 'r' to restart or 'q' to quit", frame)

        #Capture the frame when 'c' is pressed
        if cv2.waitKey(1) & 0xFF == ord('c'):
            img_path = f'training_faces/{name}/{name}_{len(os.listdir(person_folder))+1}.jpg'
            cv2.imwrite(img_path, frame) #Save the original frame
            print(f"Image saved at: {img_path}")

        #Exit loop with when pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if cv2.waitKey(1) & 0xFF ==ord('r'):
            cap.release()
            cv2.destroyAllWindows()
            return capture_images()

    cap.release()
    cv2.destroyAllWindows()


capture_images()


Encode and train based on faces

In [24]:
def process_and_save_encodings():
    base_folder = "training_faces"
    encoding_base = "encodings"
    
    os.makedirs(encoding_base, exist_ok=True)


    for person_name in os.listdir(base_folder):
        #Check if pictures of person exist
        person_folder = os.path.join(base_folder, person_name)
        if not os.path.isdir(person_folder):
            continue

        #Check if encodings folder for that person exist
        person_encoding_folder = os.path.join(encoding_base, person_name)
        os.makedirs(person_encoding_folder, exist_ok=True)


        #Exploere image files per person
        for img_file in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_file)
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            face_locs = face_recognition.face_locations(rgb_img)

            if len(face_locs) == 0:
                print(f"No face found in {img_file}, skipping.")
                continue

            encoding = face_recognition.face_encodings(rgb_img, known_face_locations=face_locs)[0]
            
            #Save encoding separately
            encoding_filename = os.path.splitext(img_file)[0] + ".npy"
            encoding_path = os.path.join(person_encoding_folder, encoding_filename)
            np.save(encoding_path, encoding)
            print(f"Saved encoding: {encoding_path}")

def load_encodings():
    encoding_base ="encodings"
    encodings = []
    names = []
    
    for person_name in os.listdir(encoding_base):
        person_folder = os.path.join(encoding_base, person_name)
        if not os.path.isdir(person_folder):
            continue

        for file in os.listdir(person_folder):
            if file.endswith(".npy"):
                encoding = np.load(os.path.join(person_folder, file))
                encodings.append(encoding)
                names.append(person_name)

    return encodings, names


In [25]:
def start_recognition():
    known_encodings, known_names = load_encodings()
    if len(known_encodings) == 0:
        return

    cap = cv2.VideoCapture(0)
    while True:
        success, frame = cap.read()
        if not success:
            break


        #Convert to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Downscaling for better performance
        scale_factor = 0.25
        small_frame = cv2.resize(rgb_frame, (0,0), fx =  scale_factor, fy = scale_factor)
        
        
        #Detect faces on the smaller frame
        face_locations = face_recognition.face_locations(small_frame, model="hog")
        face_encs = face_recognition.face_encodings(small_frame, face_locations)

        #Time to scale back
        face_locations =[(int(top/scale_factor), int(right/scale_factor),
                          int(bottom/scale_factor), int(left/scale_factor))
                          for (top, right, bottom, left) in face_locations]
        
        for (top, right, bottom, left), face_enc in zip(face_locations, face_encs):
            matches = face_recognition.compare_faces(known_encodings, face_enc, tolerance=0.5)
            name = "Unknown"

            if True in matches:
                matched_idxs = [i for i, val in enumerate(matches) if val]
                counts = {}
                for idx in matched_idxs:
                    counts[known_names[idx]] = counts.get(known_names[idx], 0) + 1
                name = max(counts, key=counts.get)

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Real-Time Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [26]:
process_and_save_encodings()

Saved encoding: encodings\JoseLuis\JoseLuis_1.npy
Saved encoding: encodings\JoseLuis\JoseLuis_2.npy
Saved encoding: encodings\JoseLuis\JoseLuis_3.npy
Saved encoding: encodings\Kiara\Kiara_1.npy
Saved encoding: encodings\Kiara\Kiara_2.npy
No face found in Kiara_3.jpg, skipping.
Saved encoding: encodings\Mafer\Mafer_1.npy
Saved encoding: encodings\Mafer\Mafer_2.npy
Saved encoding: encodings\Mafer\Mafer_3.npy


In [ ]:
start_recognition()